In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import DesiredCapabilities
import random
import time
import re
import pickle

chrome_options = Options()
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_argument('--start-maximised')
#chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')

# capabilities = DesiredCapabilities.CHROME.copy()
# capabilities['acceptSslCerts'] = True 
# capabilities['acceptInsecureCerts'] = True

timeDelay = random.randrange(5, 10)
def products_cards(category):
    url = "https://www.influenster.com/reviews/" + category + "/"
    
    try:
        myDriver = webdriver.Firefox()#('/usr/local/bin/chromedriver',chrome_options=chrome_options)
        myDriver.get(url)
        print(myDriver.get_cookies())
        cookies = pickle.load(open("cookies.pkl", "rb"))
        for cookie in cookies:
            if 'expiry' in cookie:
                del cookie['expiry']
            myDriver.add_cookie(cookie)
        #myDriver.find_element_by_xpath('//*[@id="CookielawBanner"]/div/p/a').click();
    except TimeoutException:
        print("Page expired")
        
    product_link = 'https://www.influenster.com/reviews/'
    product_list = []
    product_detail_list = []
    click_count = 0
    while click_count < 100: #50000
        try:
            product_template = WebDriverWait(myDriver, 60).until(ec.presence_of_element_located((By.CLASS_NAME, "category-products")))#myDriver.find_element_by_class_name('category-products')
            cards = product_template.find_elements_by_class_name('category-product')
            for card in cards:
                link = card.get_attribute('href')
                if link not in product_list:
                    product_list.append(link)
                    name = card.find_element_by_class_name('category-product-title').text
                    brand = card.find_element_by_class_name('category-product-brand').text
                    star_rating = card.find_element_by_class_name('avg-stars.small').get_attribute('data-stars')
                    reviews = card.find_element_by_class_name('category-product-rating').text
                    image = card.find_element_by_css_selector('img').get_attribute('src')
                    
                    product_detail_list.append({'product_link': link, 'product_name': name, 'brand': brand, 'star_rating': star_rating, 
                                                'total_reviews': re.findall(r'[0-9.,]+', reviews)[-1], 'image_url': image})

                    print('name:', name, 'link:', link, 'brand:', brand, 'star_rating:', 
                          star_rating, 'reviews:', re.findall(r'[0-9.,]+', reviews)[-1], 'image_url:', image)
            time.sleep(timeDelay)
            load_more = product_template.find_element_by_class_name("category-products-pagination-page.next")
            load_more.click()
            click_count+=1
            time.sleep(timeDelay)
            
        except Exception as e:
            print(e)
            return product_detail_list
            break
    else:
        print(click_count)
        return product_detail_list
    
try:
    # set up chromedriver
    myDriver=webdriver.Firefox()
    actions = ActionChains(myDriver)
    prod_list = products_cards("multivitamins")
    prod_data_frame = pd.DataFrame(prod_list)
    prod_data_frame.to_csv("influenster_product_list_multivitamins_2.csv", index=False)
except Exception as e:
    print(e)
finally:
    myDriver.quit()
    print('exit')

[{u'domain': u'.influenster.com', u'name': u'__cfduid', u'value': u'd3b5759c9ffa402153dc271b86ef88b681578461438', u'expiry': 1581053438, u'path': u'/', u'httpOnly': True, u'secure': False}]
('name:', u"OLLY\xae The Perfect Women's Multi-Vitamin Blissful Berry Gummies", 'link:', u'https://www.influenster.com/reviews/olly-the-perfect-womens-multi-vitamin-blissful-berry-gummies', 'brand:', u'By Olly', 'star_rating:', u'4.62285714285714', 'reviews:', u'350', 'image_url:', u'https://image.influenster.com/eyJleHRlbmQiOiB7fSwgImJ1Y2tldCI6ICJpbmZsdWVuc3Rlcl9wcm9kdWN0aW9uIiwgImtleSI6ICJtZWRpYS9wcm9kdWN0L2ltYWdlLzgxUUlNSUlMdU5MLl9TWTY3OV8uanBnIiwgImVkaXRzIjogeyJyZXNpemUiOiB7IndpZHRoIjogMzU1LCAid2l0aG91dEVubGFyZ2VtZW50IjogdHJ1ZSwgImJhY2tncm91bmQiOiB7ImFscGhhIjogMCwgImciOiAxLCAiciI6IDEsICJiIjogMX0sICJoZWlnaHQiOiAzNTUsICJmaXQiOiAiaW5zaWRlIn19fQ==')
('name:', u'vitafusion\u2122 MultiVites\u2122 Complete Multivitamin Berry Peach and Orange', 'link:', u'https://www.influenster.com/reviews/vitafusiontm

('name:', u'Optimum Nutrition Opti-Women Multivitamin', 'link:', u'https://www.influenster.com/reviews/optimum-nutrition-opti-women-multivitamin', 'brand:', u'By Optimum Nutrition', 'star_rating:', u'4.66666666666667', 'reviews:', u'60', 'image_url:', u'https://image.influenster.com/eyJleHRlbmQiOiB7fSwgImJ1Y2tldCI6ICJpbmZsdWVuc3Rlcl9wcm9kdWN0aW9uIiwgImtleSI6ICJtZWRpYS9wcm9kdWN0L2ltYWdlL2YwZmJkNmIwMDMwMmI2NzcxMTM4OWM1ZDkzN2RkZjZkLnBuZyIsICJlZGl0cyI6IHsicmVzaXplIjogeyJ3aWR0aCI6IDM1NSwgIndpdGhvdXRFbmxhcmdlbWVudCI6IHRydWUsICJiYWNrZ3JvdW5kIjogeyJhbHBoYSI6IDAsICJnIjogMSwgInIiOiAxLCAiYiI6IDF9LCAiaGVpZ2h0IjogMzU1LCAiZml0IjogImluc2lkZSJ9fX0=')
('name:', u'Healthy Indulgence Multivitamin & Minerals', 'link:', u'https://www.influenster.com/reviews/healthy-indulgence-multivitamin-minerals', 'brand:', u'By Healthy Indulgence', 'star_rating:', u'4.75', 'reviews:', u'4', 'image_url:', u'https://image.influenster.com/eyJleHRlbmQiOiB7fSwgImJ1Y2tldCI6ICJpbmZsdWVuc3Rlcl9wcm9kdWN0aW9uIiwgImtleSI6ICJtZWRpY

('name:', u'Rainbow Light Nutri Stars Chewable Multivitamin Chewable Tablets', 'link:', u'https://www.influenster.com/reviews/rainbow-light-nutri-stars-chewable-multivitamin-chewable-tablets', 'brand:', u'By Rainbow Light', 'star_rating:', u'4.86666666666667', 'reviews:', u'15', 'image_url:', u'https://image.influenster.com/eyJleHRlbmQiOiB7fSwgImtleSI6ICJtZWRpYS9wcm9kdWN0L2ltYWdlLzQ3NjQ4MjExY2U5NmJiYTI4MDEyMGZmMzg4NzRiMzQxLnBuZyIsICJlZGl0cyI6IHsicmVzaXplIjogeyJ3aXRob3V0RW5sYXJnZW1lbnQiOiB0cnVlLCAiZml0IjogImluc2lkZSIsICJiYWNrZ3JvdW5kIjogeyJnIjogMSwgInIiOiAxLCAiYWxwaGEiOiAwLCAiYiI6IDF9LCAiaGVpZ2h0IjogMzU1LCAid2lkdGgiOiAzNTV9fSwgImJ1Y2tldCI6ICJpbmZsdWVuc3Rlcl9wcm9kdWN0aW9uIn0=')
('name:', u"Smartypants Smarty Pants Women's Complete Multivitamin Gummies - 120...", 'link:', u'https://www.influenster.com/reviews/smartypants-smarty-pants-womens-complete-multivitamin-gummies-120-count', 'brand:', u'By SmartyPants', 'star_rating:', u'4.47368421052632', 'reviews:', u'38', 'image_url:', u'https:/

KeyboardInterrupt: 